<a href="https://colab.research.google.com/github/nicolycosta/Analise-de-Correlacao-e-Covariancia/blob/main/Covariance_and_Correlation_Matrix_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy

In [4]:
df = pd.read_excel("Base.xlsx")
df = df.set_index('Mês')
df.head(5)

,ICMS- Pará,IPCA - Núm índice
Mês,,
2004-01-01,2.080399e+08,2246.43
2004-02-01,1.887688e+08,2260.13
2004-03-01,1.593895e+08,2270.75
2004-04-01,1.749718e+08,2279.15
2004-05-01,1.973934e+08,2290.77


In [6]:
#Tornando a série em estacionária
df = df.diff().dropna()
df.head(5)

,ICMS- Pará,IPCA - Núm índice
Mês,,
2004-03-01,-10108213.33,-3.08
2004-04-01,44961596.48,-2.22
2004-05-01,6839299.42,3.22
2004-06-01,-27836294.21,4.64
2004-07-01,7214018.82,4.73


# Covariância(X,Y) e Autocovariância

In [17]:
def Covariancia(x,y):
    
    from statistics import mean
    import pandas as pd
    import numpy as np
    import warnings    
    warnings.filterwarnings('ignore')
    
    data = np.array([x.values, y.values])
    Cal= pd.DataFrame(data)
    Cal = Cal.T 
    Cal.columns=['X', 'Y']
    Cal

    Cal['Covx']= 'NaN'
    Cal['Covy']= 'NaN'

    #mean
    meanx= mean(Cal['X'])
    meany= mean(Cal['Y'])

    i= 0
    for j in Cal['Covx']:
        Cal['Covx'][i] = (Cal['X'][i] - meanx) # (x-μx)
        i = i + 1
    i= 0
    for j in Cal['Covy']:
        Cal['Covy'][i] = Cal['Y'][i] - meany # (y-μy)
        i = i + 1

    
    Cal['Covx*Covy'] = Cal['Covx']*Cal['Covy'] #(x-μx)(y-μy)
    covxy = mean(Cal['Covx*Covy']) # E(x-μx)(y-μy)
    covx = mean(Cal['Covx']*Cal['Covx']) # E(x-μx)(x-μx)
    covy = mean(Cal['Covy']*Cal['Covy']) # E(y-μy)(y-μy)

    print()
    print('Covariância cruzada: ')
    print(f'                  {covxy}')
    print()
    print('Auto-Covariância do X: ')
    print(f'                  {covx}')
    print()
    print('Auto-Covariância do Y: ')
    print(f'                  {covy}')

In [18]:
Covariancia(df['ICMS- Pará'],df['IPCA - Núm índice '])


Covariância cruzada: 
                  -98470499.91919743

Auto-Covariância do X: 
                  9236236617723350.0

Auto-Covariância do Y: 
                  158.9864613905312


#### Matriz de Covariância (função pronta)

In [16]:
numpy.cov(df['ICMS- Pará'], df['IPCA - Núm índice '])

array([[ 9.28085612e+15, -9.89462028e+07],
       [-9.89462028e+07,  1.59754512e+02]])